In [ ]:
import os
import glob
import subprocess as sp

In [ ]:
def generate_figure_text(respth, cmplpth, version='figure', figcap=""):
    """get all figures from respth
    respth: figure inputs
    cmplpth: compiled results output
    
    version: choose from `graph` or `figure` or `page`
    """
    assert version in ['graph', 'figure', 'page']
    
    # input
    figures = glob.glob("fig*.pdf", dir_fd=respth) 
        
    # rename
    for i, fig in enumerate(figures):
        # remove extra .
        fignew = fig
        while fignew.count('.') > 1:
            fignew = fignew.replace('.', 'p', 1)

        # update name (using symlink)
        src = os.path.join(respth, fig)
        dst = os.path.join(cmplpth, fignew)
        if not os.path.isfile(dst):
            os.symlink(src, dst) 
        figures[i] = fignew

    # prep figure text
    figuretext = ""
    for fig in figures:
        if version == 'graph':
            figuretext += f"\\centerline{{\\includegraphics[width=\\textwidth]{{{fig}}}}}\n"
        elif version == 'page':
            figuretext += f"\\includepdf[pages=-]{{{fig}}}\n"
        if version == 'figure':
            figt = f"""
            \\begin{{figure}}[ht]
                \\caption{{{figcap}}}
                \\centerline{{\\includegraphics[width=\\textwidth]{{{fig}}}}}
            \\end{{figure}}
            """
            figuretext += figt
            
    return figuretext

def generate_tex_insertpage(title, author, figuretext):
    """Insert PDF pages
    """
    latex_letter = f"""\\documentclass[11pt]{{article}}
    \\usepackage{{pdfpages}}
    \\usepackage[utf8]{{inputenc}}
    \\usepackage[legalpaper, margin=0.5in]{{geometry}}
    \\begin{{document}}
    \\title{{{title}}}
    \\author{{{author}}}
    \\maketitle
    {figuretext} % \\includepdf[pages=-]{{fig1-2_basis_space_righthalf_2022-07-20.pdf}}
    \\end{{document}}
    """ 
    return latex_letter

def generate_tex(title, author, figuretext):
    """Insert as graphics
    """
    latex_letter = f"""\\documentclass[11pt]{{article}}
    \\renewcommand{{\\familydefault}}{{\\sfdefault}}
    \\usepackage{{graphicx}}
    \\usepackage[utf8]{{inputenc}}
    \\usepackage[legalpaper, margin=0.5in]{{geometry}}
    \\begin{{document}}
    \\title{{{title}}}
    \\author{{{author}}}
    \\maketitle
    \\listoffigures
    {figuretext}
    % \\centerline{{\\includegraphics[width=\\textwidth]{{fig1-2_basis_space_righthalf_2022-07-20.pdf}}}}
    
    \\end{{document}}
    """ 
    return latex_letter

In [ ]:
basepth = '/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t5'
respth = os.path.join(basepth, 'figures')
# output
cmplpth = os.path.join(respth, 'compiled')
if not os.path.isdir(cmplpth):
    os.mkdir(cmplpth)
for f in glob.glob(os.path.join(cmplpth, '*')):
    os.remove(f)

title = 'dredFISH default analysis'
author = 'Fangming'
figuretext = generate_figure_text(respth, cmplpth)
latex_letter = generate_tex(title, author, figuretext)
# print(figuretext)
# print(latex_letter)

texpth = os.path.join(cmplpth, 'compiled.tex')
with open(texpth, 'w') as fh:
    fh.write(latex_letter)
    
sp.run(['pdflatex', 'compiled.tex'], cwd=cmplpth)

In [ ]:
sp.run(['pdflatex', 'compiled.tex'], cwd=cmplpth)